In [1]:
!pip install -U git+https://github.com/Sakib323/AI-Game-Engine.git
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip install transformers
!pip install datasets
!pip install wandb
!pip install -U datasets
!pip install objaverse
!pip install diffusers
!pip install trimesh
!pip install jaxtyping
!pip install pytorch-lightning
!pip install ijson
!pip install triton==3.2.0
!pip install wandb

  Cloning https://github.com/Sakib323/AI-Game-Engine.git to /tmp/pip-req-build-kvub_4zj
  Running command git clone --filter=blob:none --quiet https://github.com/Sakib323/AI-Game-Engine.git /tmp/pip-req-build-kvub_4zj
  Resolved https://github.com/Sakib323/AI-Game-Engine.git to commit 9f683a25e94329e2e0d9f842fc1a320321aa2abd
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.9 MB/s eta 0:00:00:00:

In [ ]:
!git clone --depth 1 --branch main https://github.com/stepfun-ai/Step1X-3D.git
import sys
sys.path.append("./Step1X-3D")  
import os
print(os.listdir("./Step1X-3D"))
!pip install -r ./Step1X-3D/requirements.txt --verbose

Cloning into 'Step1X-3D'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (400/400), done.
remote: Compressing objects: 100% (364/364), done.
remote: Total 400 (delta 24), reused 385 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (400/400), 134.62 MiB | 21.93 MiB/s, done.
Resolving deltas: 100% (24/24), done.
['examples', 'assets', 'README_CN.md', 'inference.py', 'zero_to_fp32.py', 'requirements.txt', 'data', 'LICENSE', 'train_ig2mv.sh', 'train_autoencoder.sh', 'step1x3d_geometry', '.gitignore', 'train_diffusion.sh', 'app.py', 'configs', 'train_ig2mv.py', 'README.md', 'step1x3d_texture', 'val_data', '.git', 'train.py']
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Cloning https://github.com/NVlabs/nvdiffrast.git to /tmp/pip-req-build-p10n34fy
  Running command git version
  git version 2.34.1
  Running command git clone --filter=blob:none https://github.com/NVlabs/nvdiffrast.git /tmp/pip-req-build-p10n34fy
  Cloni

In [ ]:
pip install torch-cluster -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html

# =========================================================================================
# TRAIN MESH GENERATION MODEL: DOWNLOAD DATASET & TRAIN THE MODEL
# This script download the dataset then process them and lastly train the model.
# =========================================================================================

In [1]:
import requests
import gzip
import json
import objaverse
import gc
import os
import ijson


KEYS_TO_EXCLUDE = {
    "tags", "name", "staffpickedAt", "viewCount", "likeCount", "animationCount",
    "commentCount", "publishedAt", "user", "description", "faceCount", "createdAt",
    "vertexCount", "license", "uri", "viewerUrl", "embedUrl", "isDownloadable",
    "categories", "isAgeRestricted", "archives"
}

print("Starting dataset preparation...")

# Check if captions file exists; download only if necessary
url = "https://huggingface.co/datasets/tiange/Cap3D/resolve/main/Objaverse_files/cap3d_captions.json.gz"
captions_file = 'cap3d_captions.json.gz'
if not os.path.exists(captions_file):
    print("Downloading captions file...")
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(captions_file, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Captions file downloaded successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading captions: {e}")
        exit()
else:
    print("Captions file already exists, skipping download.")

def process_batch(batch, f, total_counter):
    uids = [uid for uid, _ in batch]
    try:
        annotations = objaverse.load_annotations(uids)
    except Exception as e:
        print(f"Error loading annotations for batch: {e}")
        return 0  # Skip this batch on error

    for i, (uid, caption) in enumerate(batch):
        metadata = annotations.get(uid, {})
        filtered_metadata = {k: v for k, v in metadata.items() if k not in KEYS_TO_EXCLUDE}

        # Filter thumbnails to keep only 1024x576 images
        if "thumbnails" in filtered_metadata and "images" in filtered_metadata["thumbnails"]:
            filtered_metadata["thumbnails"]["images"] = [
                img for img in filtered_metadata["thumbnails"]["images"]
                if img.get("width") == 1024 and img.get("height") == 576
            ]

        datapoint = {
            "uid": uid,
            "caption": caption,
            **filtered_metadata
        }
        f.write(json.dumps(datapoint) + '\n')
        if total_counter + i < 10:
            print(f"Datapoint {uid}:")
            print(json.dumps(datapoint, indent=2))
            print("-" * 50)
    del annotations
    gc.collect()
    return len(batch)


batch_size = 100000
total_counter = 0
with open('extended_dataset.jsonl', 'w', encoding='utf-8') as f:
    batch = []
    for uid, caption in ijson.kvitems(gzip.open(captions_file, 'rt', encoding='utf-8'), ''):
        batch.append((uid, caption))
        if len(batch) >= batch_size:
            processed = process_batch(batch, f, total_counter)
            total_counter += processed
            print(f"Processed {total_counter} captions so far.")
            batch = []
    if batch:
        processed = process_batch(batch, f, total_counter)
        total_counter += processed

print(f"Total entries in extended dataset: {total_counter}")
print("Dataset saved to 'extended_dataset.jsonl'")
print("Script finished.")

Starting dataset preparation...
Captions file downloaded successfully.


 99%|█████████▉| 159/160 [02:01<00:00,  1.31it/s]


Datapoint ed51a51909ee46c780db3a85e821feb2:
{
  "uid": "ed51a51909ee46c780db3a85e821feb2",
  "caption": "Matte green rifle with a long barrel, stock, and detailed magazine.",
  "thumbnails": {
    "images": [
      {
        "uid": "0d6d45c0ee174cca9323a183ebbe2ef3",
        "size": 10719,
        "width": 1024,
        "url": "https://media.sketchfab.com/models/ed51a51909ee46c780db3a85e821feb2/thumbnails/f46b00b385d4449d923aff78348b04c6/eb41be4a1dcd450a974db9d24dc13d16.jpeg",
        "height": 576
      }
    ]
  }
}
--------------------------------------------------
Datapoint 9110b606f6c547b2980fcb3c8c4b6a1c:
{
  "uid": "9110b606f6c547b2980fcb3c8c4b6a1c",
  "caption": "Rustic single-story building with a weathered green gable roof, exposed wooden beams, brick walls, and a partly visible glass window front.",
  "thumbnails": {
    "images": [
      {
        "uid": "32b5631f607543a38e10188b03e3d7c1",
        "size": 80170,
        "width": 1024,
        "url": "https://media.sketchfab

  1%|▏         | 2/160 [00:00<01:12,  2.17it/s]


KeyboardInterrupt: 

In [2]:
import json
import objaverse
import pathlib
import shutil
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  

jsonl_path = "extended_dataset.jsonl"
output_dir = pathlib.Path("downloads")
output_dir.mkdir(exist_ok=True)
metadata_output_path = output_dir / "metadata.json"
download_limit = 100

uid_metadata = {}
with open(jsonl_path, "r", encoding="utf-8") as f:
    for line in f:
        if len(uid_metadata) >= download_limit:
            break
        entry = json.loads(line)
        uid = entry["uid"]
        caption = entry.get("caption", "")
        thumbnails = entry.get("thumbnails", {}).get("images", [])
        thumbnail_urls = [img["url"] for img in thumbnails if "url" in img]

        uid_metadata[uid] = {
            "caption": caption,
            "thumbnails": thumbnail_urls
        }

uids = list(uid_metadata.keys())
print(f"Downloading {len(uids)} models...")
paths = objaverse.load_objects(uids=uids)

final_metadata = {}
for uid, src_path in paths.items():
    dst_path = output_dir / f"{uid}.glb"
    try:
        shutil.copy(src_path, dst_path)
        final_metadata[uid] = uid_metadata[uid]
        print(f"Saved {uid} to {dst_path}")
    except Exception as e:
        print(f"Failed to save {uid}: {e}")

with open(metadata_output_path, "w", encoding="utf-8") as f:
    json.dump(final_metadata, f, indent=2)

print(f"\nDownloaded {len(final_metadata)} models.")
print(f"Metadata saved to {metadata_output_path}")


Downloaded 1 / 100 objects
Downloaded 2 / 100 objects
Downloaded 3 / 100 objects
Downloaded 4 / 100 objects
Downloaded 5 / 100 objects
Downloaded 6 / 100 objects
Downloaded 7 / 100 objects
Downloaded 8 / 100 objects
Downloaded 9 / 100 objects
Downloaded 10 / 100 objects
Downloaded 11 / 100 objects
Downloaded 12 / 100 objects
Downloaded 13 / 100 objects
Downloaded 14 / 100 objects
Downloaded 15 / 100 objects
Downloaded 16 / 100 objects
Downloaded 17 / 100 objects
Downloaded 18 / 100 objects
Downloaded 19 / 100 objects
Downloaded 20 / 100 objects
Downloaded 21 / 100 objects
Downloaded 22 / 100 objects
Downloaded 23 / 100 objects
Downloaded 24 / 100 objects
Downloaded 25 / 100 objects
Downloaded 26 / 100 objects
Downloaded 27 / 100 objects
Downloaded 28 / 100 objects
Downloaded 29 / 100 objects
Downloaded 30 / 100 objects
Downloaded 31 / 100 objects
Downloaded 32 / 100 objects
Downloaded 33 / 100 objects
Downloaded 34 / 100 objects
Downloaded 35 / 100 objects
Downloaded 36 / 100 objects
D

In [3]:
import requests
import os
import tempfile
import trimesh
import numpy as np
import torch
from transformers import AutoTokenizer
from tqdm import tqdm
import random
import traceback
import logging
import sys
from diffusers import AutoencoderKL


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32
print(f"Using device: {device}, dtype: {dtype}")

sys.path.append("./Step1X-3D")

from step1x3d_geometry.models.pipelines.pipeline import Step1X3DGeometryPipeline

from huggingface_hub import hf_hub_download
print("Initializing Step1X-3D VAE...")
try:
    geometry_pipeline = Step1X3DGeometryPipeline.from_pretrained(
        "stepfun-ai/Step1X-3D",
        subfolder='Step1X-3D-Geometry-1300m',
        torch_dtype=dtype
    )
    vae = geometry_pipeline.vae.to(device)
    vae.eval()
    print("Step1X-3D VAE initialized successfully.")
except Exception as e:
    print(f"Error initializing pipeline. Make sure you have the Step1X-3D repo cloned and in your sys.path. Error: {e}")
    vae = None

tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token


image_vae = AutoencoderKL.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        subfolder="vae",
        torch_dtype=dtype
    ).to(device).eval()
print("Image VAE model loaded.")


2025-06-17 12:14:10.125232: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750162450.283423     715 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750162450.330546     715 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda, dtype: torch.float32


ModuleNotFoundError: No module named 'streaming'

In [ ]:
from diffusers import AutoencoderKL
import requests
from PIL import Image
from io import BytesIO
import torch
import torchvision.transforms as transforms
from torchvision.transforms.functional import crop
from diffusers import AutoencoderKL
import torchvision.transforms as T


def get_image_latent(url: str, vae: AutoencoderKL, device: str = "cuda") -> torch.Tensor:

    try:
        response = requests.get(url)
        response.raise_for_status()
        img_data = BytesIO(response.content)
        raw_image = Image.open(img_data).convert("RGB")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading image: {e}")
        return None
    except IOError as e:
        print(f"Error opening image: {e}")
        return None

    preprocess = T.Compose([
        T.Resize(512, interpolation=T.InterpolationMode.BICUBIC),
        T.CenterCrop(512),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

    image_tensor = preprocess(raw_image).unsqueeze(0).to(device=device, dtype=vae.dtype)


    with torch.no_grad():
        latent_dist = vae.encode(image_tensor).latent_dist
        latents = latent_dist.sample()
        latents = latents * vae.config.scaling_factor

    return latents
    

In [ ]:
import os
import json
import torch
import traceback
import numpy as np
import trimesh
from tqdm import tqdm
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def sample_uniform_and_sharp_points(mesh, num_uniform_points=16384, num_sharp_points=16384, sharp_threshold_deg=30):
    """
    Performs separate uniform and sharp edge sampling on a mesh.
    """
    uniform_points, face_indices_uniform = trimesh.sample.sample_surface(mesh, num_uniform_points)
    uniform_normals = mesh.face_normals[face_indices_uniform]
    sharp_points, sharp_normals = None, None

    try:
        edge_angles = mesh.face_adjacency_angles
        sharp_threshold_rad = np.deg2rad(sharp_threshold_deg)
        sharp_edge_indices = np.where(edge_angles > sharp_threshold_rad)[0]

        if len(sharp_edge_indices) > 0:
            face_indices_of_sharp_edges = mesh.face_adjacency[sharp_edge_indices].flatten()
            sharp_face_indices = np.unique(face_indices_of_sharp_edges)
            if len(sharp_face_indices) > 0:
                sharp_mesh = mesh.submesh([sharp_face_indices], append=True)
                if sharp_mesh.vertices.shape[0] > 3 and sharp_mesh.faces.shape[0] > 1:
                    sharp_points, face_indices_sharp = trimesh.sample.sample_surface(sharp_mesh, num_sharp_points)
                    sharp_normals = sharp_mesh.face_normals[face_indices_sharp]
    except Exception as e:
        logger.warning(f"Could not perform sharp sampling due to a mesh error: {e}. Falling back to uniform.")
        sharp_points = None

    if sharp_points is None or sharp_normals is None:
        logger.warning("No sharp regions found or submesh failed. Using uniform samples for the sharp set as a fallback.")
        sharp_points, face_indices_sharp = trimesh.sample.sample_surface(mesh, num_sharp_points)
        sharp_normals = mesh.face_normals[face_indices_sharp]

    return (uniform_points, uniform_normals), (sharp_points, sharp_normals)


def process_mesh_to_vae_input(mesh_path, num_points=32768):
    try:
        ext = os.path.splitext(mesh_path)[1].lower()
        mesh = trimesh.load(mesh_path, force='mesh', process=True)
        
        if isinstance(mesh, trimesh.Scene):
            mesh = mesh.dump().sum()

        if not isinstance(mesh, trimesh.Trimesh) or len(mesh.vertices) == 0 or len(mesh.faces) == 0:
            logger.warning(f"Skipping {mesh_path}: No valid mesh data found after loading.")
            return None
            
        if not mesh.is_watertight:
            trimesh.repair.fill_holes(mesh)

        center = mesh.bounds.mean(axis=0)
        mesh.apply_translation(-center)
        max_extent = np.max(np.linalg.norm(mesh.vertices, axis=1))
        if max_extent > 1e-6:
            mesh.apply_scale(1.0 / max_extent)
            
        (uniform_points, uniform_normals), (sharp_points, sharp_normals) = sample_uniform_and_sharp_points(
            mesh, num_uniform_points=num_points, num_sharp_points=num_points
        )

        surface_cloud = np.hstack([uniform_points, uniform_normals])
        sharp_cloud = np.hstack([sharp_points, sharp_normals])

        # Create tensors with the correct global dtype (now float32)
        return {
            "surface": torch.tensor(surface_cloud, dtype=dtype).unsqueeze(0),
            "sharp_surface": torch.tensor(sharp_cloud, dtype=dtype).unsqueeze(0)
        }
    except Exception as e:
        logger.error(f"Error processing {mesh_path}: {str(e)}")
        logger.debug(traceback.format_exc())
        return None

def process_all_meshes(download_dir="downloads", metadata_file="downloads/metadata.json"):
    # Ensure global variables are accessible or passed as arguments
    global device, vae, image_vae, tokenizer 
    
    logger.info("Starting dataset creation with the updated process_all_meshes function...")
    processed_data = []
    
    if not os.path.exists(metadata_file):
        logger.error(f"Metadata file not found at: {metadata_file}")
        return []
        
    with open(metadata_file, 'r', encoding='utf-8') as f:
        metadata = json.load(f)

    for uid, info in tqdm(metadata.items(), desc="Processing Raw Data"):
        mesh_path = os.path.join(download_dir, f"{uid}.glb")
        caption = info.get('caption', '')
        
        thumbnails = info.get('thumbnails')
        if not thumbnails or not isinstance(thumbnails, list) or not thumbnails[0]:
            continue  
        image_url = thumbnails[0]

        if not os.path.isfile(mesh_path) or not image_url:
            continue

        mesh_inputs = process_mesh_to_vae_input(mesh_path, num_points=32768)
        
        if mesh_inputs is None:
            continue
        
        mesh_inputs_on_device = {
            "surface": mesh_inputs["surface"].to(device),
            "sharp_surface": mesh_inputs["sharp_surface"].to(device) if mesh_inputs.get("sharp_surface") is not None else None
        }
        
        with torch.no_grad():
            _shape_embeds, kl_embed, _posterior = vae.encode(
                sample_posterior=True, **mesh_inputs_on_device)
            latent_3d = kl_embed.squeeze(0).cpu()
            
        latent_image = get_image_latent(image_url, image_vae, device=device)
        if latent_image is None:
            continue
        latent_image = latent_image.cpu()

        tokens = tokenizer(caption, padding="max_length", max_length=128, truncation=True, return_tensors="pt")

        processed_data.append({
            "x": latent_3d,
            "y": {
                "image_latent": latent_image,
                "input_ids": tokens["input_ids"].squeeze(0),
                "attention_mask": tokens["attention_mask"].squeeze(0),
            }
        })

    logger.info(f"Successfully created dataset with {len(processed_data)} samples.")
    return processed_data


In [ ]:
!apt-get install -y libaio-dev > /dev/null

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# --- 1. Required Imports ---
import json
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import Trainer, TrainingArguments
import random
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models
import wandb


os.environ["WANDB_API_KEY"] = "89b06c10468af620747b4bd340f72fa5d56f6849"

class MeshDataset(Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

class CustomDataCollator:
    def __call__(self, features):
        batch = {}
        batch['x'] = torch.stack([f['x'] for f in features])
        y_features = [f['y'] for f in features]
        batch['y'] = {key: torch.stack([d[key] for d in y_features]) for key in y_features[0]}
        return batch

class MeshDiTTrainer(Trainer):
    def __init__(self, *args, diffusion, **kwargs):
        super().__init__(*args, **kwargs)
        self.diffusion = diffusion

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        x_start = inputs.get("x")
        model_kwargs = {"y": inputs.get("y")}
        t = torch.randint(0, self.diffusion.num_timesteps, (x_start.shape[0],), device=x_start.device).long()
        noise = torch.randn_like(x_start)
        x_t = self.diffusion.q_sample(x_start, t, noise=noise)
        
        # model's forward pass
        model_output = model(x_t, t, y=inputs['y'])
        
        # diffusion model's loss calculation
        loss_dict = self.diffusion.training_losses(model, x_start, t, model_kwargs, noise=noise)
        loss = loss_dict["loss"].mean()
        
        return (loss, loss_dict) if return_outputs else loss
    def prediction_step(
        self,
        model,
        inputs,
        prediction_loss_only,
        ignore_keys = None,
    ):
        """
        Perform an evaluation step on the model.
        """
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            loss = self.compute_loss(model, inputs)
            
        return (loss.detach(), None, None)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token

print("Initializing MeshDiT model for training...")
model = MeshDiT_models['MeshDiT-S'](
    vocab_size=tokenizer.vocab_size, image_latent_channels=4,
    image_latent_height=64, image_latent_width=64,
    use_rope=True, use_ternary_rope=False
).to(device, dtype=dtype) 

print("Initializing Gaussian diffusion process...")
diffusion = GaussianDiffusion(
    betas=get_named_beta_schedule("linear", 1000),
    model_mean_type=ModelMeanType.EPSILON,
    model_var_type=ModelVarType.FIXED_SMALL,
    loss_type=LossType.MSE,
)


all_data = process_all_meshes(download_dir="downloads", metadata_file="downloads/metadata.json")


if all_data:
    random.shuffle(all_data)
    eval_size = int(len(all_data) * 0.05)
    if eval_size == 0 and len(all_data) > 1:
        eval_size = 1
    
    train_data = all_data[eval_size:]
    eval_data = all_data[:eval_size]
    print(f"Data split: {len(train_data)} training samples, {len(eval_data)} evaluation samples.")
    train_dataset = MeshDataset(train_data)
    eval_dataset = MeshDataset(eval_data)
    data_collator = CustomDataCollator()
    
    training_args = TrainingArguments(
    output_dir="./mesh_dit_checkpoint",
    num_train_epochs=200,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False,
    max_grad_norm=1.0,
    dataloader_num_workers=2,
    remove_unused_columns=False,
    report_to=["wandb", "tensorboard"],
    run_name="MeshDiT-S-training-float32",
    )

    
    trainer = MeshDiTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        diffusion=diffusion,
    )
    print("\n" + "="*50)
    print("      Starting MeshDiT Model Training")
    print("="*50 + "\n")
    trainer.train()
    print("\nTraining complete. Saving the final model.")
    trainer.save_model("./mesh_dit_final")
    tokenizer.save_pretrained("./mesh_dit_final")
    print("Model saved to ./mesh_dit_final")
else:
    print("FATAL: Dataset creation failed. No data to train on.")

# =======================================================================================================
# THIS IS DEMO TRAINER FOR TRAINING THE MODEL WITH DEMO DATAPOINT WHILE AVOIDING DATASET PROCESSING
# =======================================================================================================

In [ ]:
import os
import json
import torch
import random
from torch.utils.data import Dataset
from tqdm import tqdm
from transformers import Trainer, TrainingArguments, AutoTokenizer
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models
import wandb

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_API_KEY"] = "89b06c10468af620747b4bd340f72fa5d56f6849"
def process_all_meshes_demo(num_samples, tokenizer):
    """
    Generates a list of random dummy data points with the same shape and
    structure as the output of the original `process_all_meshes` function.
    """
    print(f"Generating {num_samples} dummy data samples for testing...")
    all_data = []
    x_shape = (1024, 768)
    image_latent_shape = (4, 64, 64)
    text_shape = (128,)

    for _ in range(num_samples):
        sample = {
            "x": torch.randn(x_shape, dtype=torch.float32),
            "y": {
                "image_latent": torch.randn(image_latent_shape, dtype=torch.float32),
                "input_ids": torch.randint(0, tokenizer.vocab_size, text_shape, dtype=torch.long),
                "attention_mask": torch.ones(text_shape, dtype=torch.long)
            }
        }
        all_data.append(sample)
    print("Dummy data generation complete.")
    return all_data


# --- 2. Dataset, Collator, and Corrected Trainer Class ---

class MeshDataset(Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

class CustomDataCollator:
    def __call__(self, features):
        batch = {}
        batch['x'] = torch.stack([f['x'] for f in features])
        y_features = [f['y'] for f in features]
        batch['y'] = {key: torch.stack([d[key] for d in y_features]) for key in y_features[0]}
        return batch

class MeshDiTTrainer(Trainer):
    def __init__(self, *args, diffusion, **kwargs):
        super().__init__(*args, **kwargs)
        self.diffusion = diffusion

    # MODIFIED: Added **kwargs to the function signature
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        x_start = inputs.get("x")
        model_kwargs = {"y": inputs.get("y")}
        t = torch.randint(0, self.diffusion.num_timesteps, (x_start.shape[0],), device=x_start.device).long()
        noise = torch.randn_like(x_start)
        x_t = self.diffusion.q_sample(x_start, t, noise=noise)
        
        # model's forward pass
        model_output = model(x_t, t, y=inputs['y'])
        
        # diffusion model's loss calculation
        loss_dict = self.diffusion.training_losses(model, x_start, t, model_kwargs, noise=noise)
        loss = loss_dict["loss"].mean()
        
        return (loss, loss_dict) if return_outputs else loss

    def prediction_step(
        self,
        model,
        inputs,
        prediction_loss_only,
        ignore_keys = None,
    ):
        """
        Perform an evaluation step on the model.
        """
        inputs = self._prepare_inputs(inputs)
        with torch.no_grad():
            loss = self.compute_loss(model, inputs)
            
        return (loss.detach(), None, None)


# --- 3. Main Setup and Execution ---

# Setup device and data type
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token

# Generate dummy data
all_data = process_all_meshes_demo(num_samples=100, tokenizer=tokenizer)

# Initialize Model
print("Initializing MeshDiT model for training...")
model = MeshDiT_models['MeshDiT-S'](
    vocab_size=tokenizer.vocab_size, image_latent_channels=4,
    image_latent_height=64, image_latent_width=64,
    use_rope=True, use_ternary_rope=False
).to(device, dtype=dtype)

# Initialize Diffusion Process
print("Initializing Gaussian diffusion process...")
diffusion = GaussianDiffusion(
    betas=get_named_beta_schedule("linear", 1000),
    model_mean_type=ModelMeanType.EPSILON,
    model_var_type=ModelVarType.FIXED_SMALL,
    loss_type=LossType.MSE,
)

if all_data:
    # Split data and create datasets
    random.shuffle(all_data)
    train_data = all_data[5:]
    eval_data = all_data[:5]
    print(f"Data split: {len(train_data)} training samples, {len(eval_data)} evaluation samples.")
    train_dataset = MeshDataset(train_data)
    eval_dataset = MeshDataset(eval_data)
    data_collator = CustomDataCollator()

    # Configure training arguments
    training_args = TrainingArguments(
        output_dir="./mesh_dit_test_checkpoint",
        num_train_epochs=5,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        learning_rate=1e-4,
        logging_strategy="steps",
        logging_steps=1,
        eval_strategy="epoch",
        save_strategy="epoch",
        fp16=False,
        report_to="wandb",
        run_name="MeshDiT-S-test-run-fixed",
    )

    # Initialize Trainer
    trainer = MeshDiTTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        diffusion=diffusion,
    )

    # Start Training
    print("\n" + "="*50)
    print("      Starting MeshDiT Model Training Test")
    print("="*50 + "\n")
    trainer.train()
    print("\n--- Test training finished successfully! ---")
else:
    print("FATAL: Dummy data creation failed.")

# =========================================================================================
# MESH GENERATION SCRIPT: LOAD MODEL AND GENERATE MESH
# This script loads your trained model and generates a mesh.
# =========================================================================================

In [ ]:
import torch
import numpy as np
import trimesh
from PIL import Image
import requests
from io import BytesIO
from transformers import AutoTokenizer
from diffusers import AutoencoderKL
from tqdm import tqdm
import os 
from safetensors.torch import load_file
import torchvision.transforms as transforms
import sys

# --- Local Imports ---
from step1x3d_geometry.models.pipelines.pipeline import Step1X3DGeometryPipeline
from mmfreelm.models.hgrn_bit.mesh_dit import MeshDiT_models
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule, _extract_into_tensor


print("--- Starting Inference Setup ---")

# --- 1. Configuration ---
MODEL_PATH = "./mesh_dit_final"
OUTPUT_FILENAME = "generated_mesh_final.glb"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE = torch.float32

TEXT_PROMPT = "a green rifle with a long barrel"
IMAGE_URL = "https://media.sketchfab.com/models/ed51a51909ee46c780db3a85e821feb2/thumbnails/f46b00b385d4449d923aff78348b04c6/eb41be4a1dcd450a974db9d24dc13d16.jpeg"

CFG_SCALE_TEXT = 7.5
CFG_SCALE_IMAGE = 1.5
NUM_SAMPLING_STEPS = 250

# --- 2. Load All Models & Components ---
print(f"Loading models onto {DEVICE}...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
print("-> Tokenizer loaded.")

generation_model = MeshDiT_models['MeshDiT-S'](
    input_tokens=2048,
    input_dim=64,
    vocab_size=tokenizer.vocab_size,
    image_latent_channels=4,
    image_latent_height=64,
    image_latent_width=64,
    use_rope=True,
    use_ternary_rope=False
).to(DEVICE, dtype=DTYPE).eval()

state_dict = load_file(os.path.join(MODEL_PATH, "model.safetensors"), device=DEVICE)
generation_model.load_state_dict(state_dict)
print("-> Custom DiT model loaded successfully.")

geometry_pipeline = Step1X3DGeometryPipeline.from_pretrained("stepfun-ai/Step1X-3D", subfolder='Step1X-3D-Geometry-1300m', torch_dtype=DTYPE)
vae_3d = geometry_pipeline.vae.to(DEVICE).eval()
print("-> 3D VAE loaded.")

vae_image = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="vae", torch_dtype=DTYPE).to(DEVICE).eval()
print("-> Image VAE loaded.")

diffusion = GaussianDiffusion(betas=get_named_beta_schedule("linear", 1000), model_mean_type=ModelMeanType.EPSILON, model_var_type=ModelVarType.FIXED_SMALL, loss_type=LossType.MSE)
print("--- All components loaded successfully! ---")

# --- 3. Prepare Inputs for the Model ---
print("\n--- Preparing Generation Inputs ---")

tokens = tokenizer(TEXT_PROMPT, padding="max_length", max_length=128, truncation=True, return_tensors="pt")
input_ids = tokens["input_ids"].to(DEVICE)
attention_mask = tokens["attention_mask"].to(DEVICE)

def get_image_latent(url: str, vae: AutoencoderKL, device: str) -> torch.Tensor:
    response = requests.get(url)
    response.raise_for_status()
    img_data = BytesIO(response.content)
    raw_image = Image.open(img_data).convert("RGB")
    
    transform = transforms.Compose([
        transforms.Resize(512),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])
    image_tensor = transform(raw_image).unsqueeze(0).to(device=device, dtype=vae.dtype)
    with torch.no_grad():
        image_latent = vae.encode(image_tensor).latent_dist.sample() * vae.config.scaling_factor
    return image_latent

image_latent = get_image_latent(IMAGE_URL, vae_image, DEVICE)
print(f"-> Conditioning image latent created with shape: {image_latent.shape}")

model_kwargs = {"y": {"image_latent": image_latent, "input_ids": input_ids, "attention_mask": attention_mask}}

# --- 4. Generate the 3D Latent via Denoising ---
print("\n--- Starting Denoising Process ---")

z = torch.randn(1, 2048, 64, device=DEVICE, dtype=DTYPE) 

ddim_timesteps = np.asarray(list(range(0, 1000, 1000 // NUM_SAMPLING_STEPS)))
ddim_steps = torch.from_numpy(ddim_timesteps).long().to(DEVICE)

with torch.no_grad():
    for i in tqdm(range(NUM_SAMPLING_STEPS - 1, -1, -1), desc="DDIM Sampling"):
        t = ddim_steps[i].expand(z.shape[0])
        
        z_in = torch.cat([z, z], dim=0)
        t_in = torch.cat([t, t], dim=0)
        y_in = {
            "image_latent": torch.cat([model_kwargs['y']['image_latent']] * 2, dim=0),
            "input_ids": torch.cat([model_kwargs['y']['input_ids']] * 2, dim=0),
            "attention_mask": torch.cat([model_kwargs['y']['attention_mask']] * 2, dim=0)
        }
        
        noise_pred = generation_model.forward_with_cfg(
            z_in, t_in, y_in, CFG_SCALE_TEXT, CFG_SCALE_IMAGE
        )
        
        alpha_t = _extract_into_tensor(diffusion.alphas_cumprod, t, z.shape)
        t_prev_idx = ddim_steps[i - 1] if i > 0 else torch.tensor([-1], device=DEVICE, dtype=torch.long)
        alpha_t_prev = _extract_into_tensor(diffusion.alphas_cumprod_prev, t, z.shape)

        pred_x0 = (z - (1 - alpha_t).sqrt() * noise_pred) / alpha_t.sqrt()
        dir_xt = (1 - alpha_t_prev).sqrt() * noise_pred
        z = alpha_t_prev.sqrt() * pred_x0 + dir_xt

generated_latent_seq = z
print("--- Denoising complete. ---")

# --- 5. Decode Latent, Add Texture, and Save Mesh ---
print("\n--- Decoding Latent and Preparing Mesh ---")
with torch.no_grad():
    decoded_latents = vae_3d.decode(generated_latent_seq)
    
    # MODIFIED: Pass the required arguments to extract_geometry
    mesh_result = vae_3d.extract_geometry(
        decoded_latents,
        mc_level=0.5,
        bounds=[-1, -1, -1, 1, 1, 1],
        octree_resolution=256
    )[0]

# MODIFIED: Convert the MeshExtractResult into a trimesh.Trimesh object
final_mesh = trimesh.Trimesh(
    vertices=mesh_result.verts.cpu().numpy(),
    faces=mesh_result.faces.cpu().numpy()
)

print("-> Applying white texture to the mesh...")
white_texture = Image.new('RGB', (128, 128), (255, 255, 255))
material = trimesh.visual.texture.SimpleMaterial(image=white_texture)
final_mesh.visual.material = material
print("-> Texture applied successfully.")

final_mesh.export(OUTPUT_FILENAME)
print(f"\n--- ✨ Success! Mesh saved to {OUTPUT_FILENAME} ---")


In [ ]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
from diffusers import AutoencoderKL
from diffusion_model import GaussianDiffusion, ModelMeanType, ModelVarType, LossType, get_named_beta_schedule
from timm.models.vision_transformer import PatchEmbed
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F

# --- Config ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


image_size = 512
latent_downscale = 8
latent_size = image_size // latent_downscale  # 64
patch_size = 4
patch_dim = 4 * patch_size * patch_size      # 64
embed_dim = 1024
batch_size = 8

vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-2-1", subfolder="vae").to(device).eval()
betas = get_named_beta_schedule("linear", 1000)
diffusion = GaussianDiffusion(betas=betas, model_mean_type=ModelMeanType.EPSILON, model_var_type=ModelVarType.FIXED_SMALL, loss_type=LossType.MSE)
patch_embed = PatchEmbed(img_size=latent_size, patch_size=patch_size, in_chans=4, embed_dim=embed_dim).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained("Sakib323/MMfreeLM-370M")
tokenizer.pad_token = tokenizer.eos_token


transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])



def preprocess(example):
    # Image -> latent
    image = transform(example['image'].convert("RGB")).unsqueeze(0).to(device)
    with torch.no_grad():
        latents = vae.encode(image).latent_dist.sample() * vae.config.scaling_factor
        latents = F.interpolate(latents, size=(latent_size, latent_size), mode='bilinear')
    # Add noise
    t = torch.randint(0, 1000, (1,), device=device).long()
    noise = torch.randn_like(latents)
    x_noisy = diffusion.q_sample(x_start=latents, t=t, noise=noise)
    # Flatten patches via unfold: [1,4,64,64] -> [1,256,64]
    patches = F.unfold(x_noisy, kernel_size=patch_size, stride=patch_size)  # [1,64,256]
    patches = patches.permute(0, 2, 1).squeeze(0)  # [256,64]
    # Tokenize text
    tokens = tokenizer(example['text'], padding='max_length', truncation=True,
                       max_length=2048, return_tensors='pt')
    return {
        'patch_embeddings': patches.cpu(),
        'noise': noise.squeeze(0).cpu(),
        'timestep': t.item(),
        'input_ids': tokens['input_ids'].squeeze(0),
        'attention_mask': tokens['attention_mask'].squeeze(0)
    }




dataset = load_dataset("iamkaikai/GAME-MAP-ART", split="train").select(range(10))
processed = [preprocess(ex) for ex in dataset]



# Dataset and DataLoader
class DiTDataset(torch.utils.data.Dataset):
    def __init__(self, data): self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

def collate_fn(batch):
    return {
        'patch_embeddings': torch.stack([x['patch_embeddings'] for x in batch]),
        'noise': torch.stack([x['noise'] for x in batch]),
        'timestep': torch.tensor([x['timestep'] for x in batch], dtype=torch.long),
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
    }

dataloader = DataLoader(DiTDataset(processed), batch_size=batch_size,
                        shuffle=True, num_workers=4, collate_fn=collate_fn)

In [ ]:
"""
A minimal training script for DiT using PyTorch DDP.
"""
import torch
# the first flag below was False when we tested this script but True makes A100 training a lot faster:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
from torchvision.datasets import ImageFolder
from torchvision import transforms
import numpy as np
from collections import OrderedDict
from PIL import Image
from copy import deepcopy
from glob import glob
from time import time
import argparse
import logging
import os

from mmfreelm.models.hgrn_bit.ternary_dit import DiT_models
from diffusion import create_diffusion
from diffusers.models import AutoencoderKL


#################################################################################
#                             Training Helper Functions                         #
#################################################################################

@torch.no_grad()
def update_ema(ema_model, model, decay=0.9999):
    """
    Step the EMA model towards the current model.
    """
    ema_params = OrderedDict(ema_model.named_parameters())
    model_params = OrderedDict(model.named_parameters())

    for name, param in model_params.items():
        # TODO: Consider applying only to params that require_grad to avoid small numerical changes of pos_embed
        ema_params[name].mul_(decay).add_(param.data, alpha=1 - decay)


def requires_grad(model, flag=True):
    """
    Set requires_grad flag for all parameters in a model.
    """
    for p in model.parameters():
        p.requires_grad = flag


def cleanup():
    """
    End DDP training.
    """
    dist.destroy_process_group()


def create_logger(logging_dir):
    """
    Create a logger that writes to a log file and stdout.
    """
    if dist.get_rank() == 0:  # real logger
        logging.basicConfig(
            level=logging.INFO,
            format='[\033[34m%(asctime)s\033[0m] %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S',
            handlers=[logging.StreamHandler(), logging.FileHandler(f"{logging_dir}/log.txt")]
        )
        logger = logging.getLogger(__name__)
    else:  # dummy logger (does nothing)
        logger = logging.getLogger(__name__)
        logger.addHandler(logging.NullHandler())
    return logger


def center_crop_arr(pil_image, image_size):
    """
    Center cropping implementation from ADM.
    https://github.com/openai/guided-diffusion/blob/8fb3ad9197f16bbc40620447b2742e13458d2831/guided_diffusion/image_datasets.py#L126
    """
    while min(*pil_image.size) >= 2 * image_size:
        pil_image = pil_image.resize(
            tuple(x // 2 for x in pil_image.size), resample=Image.BOX
        )

    scale = image_size / min(*pil_image.size)
    pil_image = pil_image.resize(
        tuple(round(x * scale) for x in pil_image.size), resample=Image.BICUBIC
    )

    arr = np.array(pil_image)
    crop_y = (arr.shape[0] - image_size) // 2
    crop_x = (arr.shape[1] - image_size) // 2
    return Image.fromarray(arr[crop_y: crop_y + image_size, crop_x: crop_x + image_size])


#################################################################################
#                                  Training Loop                                #
#################################################################################

def main(args):
    """
    Trains a new DiT model.
    """
    assert torch.cuda.is_available(), "Training currently requires at least one GPU."

    # Setup DDP:
    dist.init_process_group("nccl")
    assert args.global_batch_size % dist.get_world_size() == 0, f"Batch size must be divisible by world size."
    rank = dist.get_rank()
    device = rank % torch.cuda.device_count()
    seed = args.global_seed * dist.get_world_size() + rank
    torch.manual_seed(seed)
    torch.cuda.set_device(device)
    print(f"Starting rank={rank}, seed={seed}, world_size={dist.get_world_size()}.")

    # Setup an experiment folder:
    if rank == 0:
        os.makedirs(args.results_dir, exist_ok=True)  # Make results folder (holds all experiment subfolders)
        experiment_index = len(glob(f"{args.results_dir}/*"))
        model_string_name = args.model.replace("/", "-")  # e.g., DiT-XL/2 --> DiT-XL-2 (for naming folders)
        experiment_dir = f"{args.results_dir}/{experiment_index:03d}-{model_string_name}"  # Create an experiment folder
        checkpoint_dir = f"{experiment_dir}/checkpoints"  # Stores saved model checkpoints
        os.makedirs(checkpoint_dir, exist_ok=True)
        logger = create_logger(experiment_dir)
        logger.info(f"Experiment directory created at {experiment_dir}")
    else:
        logger = create_logger(None)

    # Create model:
    assert args.image_size % 8 == 0, "Image size must be divisible by 8 (for the VAE encoder)."
    latent_size = args.image_size // 8
    model = DiT_models[args.model](
        input_size=latent_size,
        num_classes=args.num_classes
    )
    # Note that parameter initialization is done within the DiT constructor
    ema = deepcopy(model).to(device)  # Create an EMA of the model for use after training
    requires_grad(ema, False)
    model = DDP(model.to(device), device_ids=[rank])
    diffusion = create_diffusion(timestep_respacing="")  # default: 1000 steps, linear noise schedule
    vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-{args.vae}").to(device)
    logger.info(f"DiT Parameters: {sum(p.numel() for p in model.parameters()):,}")

    # Setup optimizer (we used default Adam betas=(0.9, 0.999) and a constant learning rate of 1e-4 in our paper):
    opt = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0)

    # Setup data:
    transform = transforms.Compose([
        transforms.Lambda(lambda pil_image: center_crop_arr(pil_image, args.image_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True)
    ])
    dataset = ImageFolder(args.data_path, transform=transform)
    sampler = DistributedSampler(
        dataset,
        num_replicas=dist.get_world_size(),
        rank=rank,
        shuffle=True,
        seed=args.global_seed
    )
    loader = DataLoader(
        dataset,
        batch_size=int(args.global_batch_size // dist.get_world_size()),
        shuffle=False,
        sampler=sampler,
        num_workers=args.num_workers,
        pin_memory=True,
        drop_last=True
    )
    logger.info(f"Dataset contains {len(dataset):,} images ({args.data_path})")

    # Prepare models for training:
    update_ema(ema, model.module, decay=0)  # Ensure EMA is initialized with synced weights
    model.train()  # important! This enables embedding dropout for classifier-free guidance
    ema.eval()  # EMA model should always be in eval mode

    # Variables for monitoring/logging purposes:
    train_steps = 0
    log_steps = 0
    running_loss = 0
    start_time = time()

    logger.info(f"Training for {args.epochs} epochs...")
    for epoch in range(args.epochs):
        sampler.set_epoch(epoch)
        logger.info(f"Beginning epoch {epoch}...")
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            with torch.no_grad():
                # Map input images to latent space + normalize latents:
                x = vae.encode(x).latent_dist.sample().mul_(0.18215)
            t = torch.randint(0, diffusion.num_timesteps, (x.shape[0],), device=device)
            model_kwargs = dict(y=y)
            loss_dict = diffusion.training_losses(model, x, t, model_kwargs)
            loss = loss_dict["loss"].mean()
            opt.zero_grad()
            loss.backward()
            opt.step()
            update_ema(ema, model.module)

            # Log loss values:
            running_loss += loss.item()
            log_steps += 1
            train_steps += 1
            if train_steps % args.log_every == 0:
                # Measure training speed:
                torch.cuda.synchronize()
                end_time = time()
                steps_per_sec = log_steps / (end_time - start_time)
                # Reduce loss history over all processes:
                avg_loss = torch.tensor(running_loss / log_steps, device=device)
                dist.all_reduce(avg_loss, op=dist.ReduceOp.SUM)
                avg_loss = avg_loss.item() / dist.get_world_size()
                logger.info(f"(step={train_steps:07d}) Train Loss: {avg_loss:.4f}, Train Steps/Sec: {steps_per_sec:.2f}")
                # Reset monitoring variables:
                running_loss = 0
                log_steps = 0
                start_time = time()

            # Save DiT checkpoint:
            if train_steps % args.ckpt_every == 0 and train_steps > 0:
                if rank == 0:
                    checkpoint = {
                        "model": model.module.state_dict(),
                        "ema": ema.state_dict(),
                        "opt": opt.state_dict(),
                        "args": args
                    }
                    checkpoint_path = f"{checkpoint_dir}/{train_steps:07d}.pt"
                    torch.save(checkpoint, checkpoint_path)
                    logger.info(f"Saved checkpoint to {checkpoint_path}")
                dist.barrier()

    model.eval()  # important! This disables randomized embedding dropout
    # do any sampling/FID calculation/etc. with ema (or model) in eval mode ...

    logger.info("Done!")
    cleanup()


if __name__ == "__main__":
    # Default args here will train DiT-XL/2 with the hyperparameters we used in our paper (except training iters).
    parser = argparse.ArgumentParser()
    parser.add_argument("--data-path", type=str, required=True)
    parser.add_argument("--results-dir", type=str, default="results")
    parser.add_argument("--model", type=str, choices=list(DiT_models.keys()), default="DiT-XL/2")
    parser.add_argument("--image-size", type=int, choices=[256, 512], default=256)
    parser.add_argument("--num-classes", type=int, default=1000)
    parser.add_argument("--epochs", type=int, default=1400)
    parser.add_argument("--global-batch-size", type=int, default=256)
    parser.add_argument("--global-seed", type=int, default=0)
    parser.add_argument("--vae", type=str, choices=["ema", "mse"], default="ema")  # Choice doesn't affect training
    parser.add_argument("--num-workers", type=int, default=4)
    parser.add_argument("--log-every", type=int, default=100)
    parser.add_argument("--ckpt-every", type=int, default=50_000)
    args = parser.parse_args()
    main(args)